In [1]:
import pandas as pd

In [2]:
customers_df = pd.read_csv('customers.csv')
products_df = pd.read_csv('products.csv')
transactions_df = pd.read_csv('transactions.csv')

### Recherche des données manquantes

In [3]:
customers_df.isnull().values.any()

False

In [4]:
products_df.isnull().values.any()

False

In [5]:
transactions_df.isnull().values.any()

False

#### Nous ne remarquons aucune donnée manquante.

### Nettoyage de données

In [6]:
# Jointure externe totale car nous souhaitons garder toutes les informations liées aux 2 dataframes.
tran_prod_df = pd.merge(transactions_df, products_df, on='id_prod', how='outer')

In [7]:
# Jointure externe gauche car nous souhaitons garder toutes les lignes de tran_prod_df
tot_df = pd.merge(tran_prod_df, customers_df, on='client_id', how='left')

In [8]:
# Récupération des Nan selon categ
tp_miss_cat = tot_df[tot_df.categ.isnull() == True]
tp_miss_cat

,id_prod,date,session_id,client_id,price,categ,sex,birth
307991,0_2245,2021-06-17 03:03:12.668129,s_49705,c_1533,NaN,NaN,m,1972.0
307992,0_2245,2021-06-16 05:53:01.627491,s_49323,c_7954,NaN,NaN,m,1973.0
307993,0_2245,2021-11-24 17:35:59.911427,s_124474,c_5120,NaN,NaN,f,1975.0
307994,0_2245,2022-02-28 18:08:49.875709,s_172304,c_4964,NaN,NaN,f,1982.0
307995,0_2245,2021-03-01 00:09:29.301897,s_3,c_580,NaN,NaN,m,1988.0
...,...,...,...,...,...,...,...,...
308089,0_2245,2021-04-06 19:59:19.462288,s_16936,c_4167,NaN,NaN,f,1979.0
308090,0_2245,2021-03-30 23:29:02.347672,s_13738,c_7790,NaN,NaN,f,1983.0
308091,0_2245,2021-12-03 14:14:40.444177,s_128815,c_6189,NaN,NaN,f,1984.0
308092,0_2245,2021-04-27 18:58:47.703374,s_26624,c_1595,NaN,NaN,f,1973.0


#### Un produit unique dont le prix n'a pas été répertorié dans les transactions. Il doit être de catégorie 0 car ce produit commence par '0'. Il s'agit ici de transactions concernant ce produit, nous pourrons remplacer les prix NaN par le prix moyen.

In [9]:
# Remplacement des categ NaN par categ 0 pour tran_prod_df
tot_df['categ'] = tot_df.categ.fillna(0)
# Remplacement des prix NaN par le prix moyen
tot_df['price'] = tot_df.price.fillna(tot_df.price.mean())

In [11]:
# Récupération des NaN selon session_id
tp_miss_ses = tot_df[tot_df.session_id.isnull() == True]
tp_miss_ses

,id_prod,date,session_id,client_id,price,categ,sex,birth
337016,0_1016,NaN,NaN,NaN,35.06,0.0,NaN,NaN
337017,0_1780,NaN,NaN,NaN,1.67,0.0,NaN,NaN
337018,0_1062,NaN,NaN,NaN,20.08,0.0,NaN,NaN
337019,0_1119,NaN,NaN,NaN,2.99,0.0,NaN,NaN
337020,0_1014,NaN,NaN,NaN,1.15,0.0,NaN,NaN
337021,1_0,NaN,NaN,NaN,31.82,1.0,NaN,NaN
337022,0_1318,NaN,NaN,NaN,20.92,0.0,NaN,NaN
337023,0_1800,NaN,NaN,NaN,22.05,0.0,NaN,NaN
337024,0_1645,NaN,NaN,NaN,2.99,0.0,NaN,NaN
337025,0_322,NaN,NaN,NaN,2.99,0.0,NaN,NaN


#### Il s'agit de 22 produits uniques qui n'ont pas été achetés car aucune donnée concernant la date, la session ou encore le client. N'ayant aucun moyen de connaître ces valeurs, il est préférable de supprimer ces 22 lignes de l'analyse.

In [12]:
# Suppression des 22 lignes sus-mentionnées.
tot_df = tot_df.dropna()

In [13]:
a = tot_df[tot_df['id_prod'] == 'T_0']
index_a = a.index
a

,id_prod,date,session_id,client_id,price,categ,sex,birth
226628,T_0,test_2021-03-01 02:30:02.237420,s_0,ct_1,-1.0,0.0,m,2001.0
226629,T_0,test_2021-03-01 02:30:02.237446,s_0,ct_1,-1.0,0.0,m,2001.0
226630,T_0,test_2021-03-01 02:30:02.237414,s_0,ct_1,-1.0,0.0,m,2001.0
226631,T_0,test_2021-03-01 02:30:02.237441,s_0,ct_0,-1.0,0.0,f,2001.0
226632,T_0,test_2021-03-01 02:30:02.237434,s_0,ct_1,-1.0,0.0,m,2001.0
...,...,...,...,...,...,...,...,...
226823,T_0,test_2021-03-01 02:30:02.237445,s_0,ct_0,-1.0,0.0,f,2001.0
226824,T_0,test_2021-03-01 02:30:02.237423,s_0,ct_1,-1.0,0.0,m,2001.0
226825,T_0,test_2021-03-01 02:30:02.237421,s_0,ct_1,-1.0,0.0,m,2001.0
226826,T_0,test_2021-03-01 02:30:02.237431,s_0,ct_1,-1.0,0.0,m,2001.0


#### Il ne s'agit pas d'achat par des clients, mais de transactions test, certainement pour vérifier que les transactions en lignes fonctionnent bien. Nous supprimerons donc ces lignes.

In [14]:
data = tot_df.drop(index_a, 0)

In [15]:
print(data.duplicated().sum())

0


#### Pas de doublons pour notre dataframe final.